## Here I translate GCN in tensorflow to GCN in tensorflow.keras


In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers
import conv_models
import tensorflow.keras.backend as K

In [ ]:
# tf and tf keras versions
print(tf.VERSION)
print(tf.keras.__version__)

In [ ]:
# load the input data and test set

adj = np.load('/Users/b_eebs/tf-keras/ZINC/adj/0.npy').astype('float32')
features = np.load('/Users/b_eebs/tf-keras/ZINC/features/0.npy').astype('float32')
test = (np.load('/Users/b_eebs/tf-keras/ZINC/logP.npy')[0:10000]).astype('float32') #takes the first 10,000 molecules

In [4]:
class convLayer(layers.Layer):

  def __init__(self, output_dim, adjacency, batch_size, **kwargs):
    self.output_dim = output_dim
    self.adjacency = adjacency # adj shape is [should be batch size , 50, 50]
    self.batch_size = batch_size # hyper parameter 
    super(convLayer, self).__init__(**kwargs)

  def build(self, input_shape):
    shape = tf.TensorShape((input_shape[1], self.output_dim))
    shape = [int(shape[0]),int(shape[1])] # [50 , 32]

    self.kernel = self.add_weight(name='kernel',
                                  shape=shape,
                                  initializer='glorot_uniform',
                                  trainable=True,dtype=tf.float32)

    self.bias = self.add_weight(name='bias',
                        shape=[shape[1]],
                        initializer='glorot_uniform',
                        trainable=True,dtype=tf.float32)

    super(convLayer, self).build(input_shape)

  def call(self, inputs):
    _features = conv_models.GCN(inputs,self.adjacency,self.kernel,self.bias) #[ batch_size, 50, 32]
    _features = tf.reshape(_features, [batch_size, -1]) # [batch_size 1600], fl. 32
    _features = tf.cast(_features, tf.float64) # [batch_size 1600], fl. 64
    return _features

  def compute_output_shape(self, input_shape):
    shape = tf.TensorShape(input_shape).as_list()
    shape[-1] = self.output_dim
    return tf.TensorShape(shape)

  def get_config(self):
    base_config = super(convLayer, self).get_config()
    base_config['output_dim'] = self.output_dim
    return base_config

  @classmethod
  def from_config(cls, config):
    return cls(**config)

In [6]:
conv_output = 32
K.set_learning_phase(1)
batch_size = 100

# define the model, composed of one layer defined in convLayers class
model = tf.keras.Sequential([convLayer(32,adj[:100],batch_size), 
                            layers.Activation('relu')])

# model compiler that optimizes and monters the mse and mae per batch
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.001),
              loss='mse',
              metrics=['mae'])


# Took 100 molecules and trained over 3 epochs with batch size = 100. Weird thing is,  number of molecules
# and the batch_size have to be the same or else it wont work. Look into this. ALSO will fail if there are no
# batches.
model.fit(features[:100], test[:100], batch_size = batch_size, epochs = 3)
model.summary()

Epoch 1/3
100/100 [==============================] - 138s 1s/sample - loss: 45.1346 - mean_absolute_error: 4.3694
Epoch 2/3
100/100 [==============================] - 149s 1s/sample - loss: 42.8778 - mean_absolute_error: 4.2704
Epoch 3/3
100/100 [==============================] - 83s 832ms/sample - loss: 40.3183 - mean_absolute_error: 4.1556
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_layer_1 (convLayer)     multiple                  1632      
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
Total params: 1,632
Trainable params: 1,632
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Here is my attempt at implementing 6 conv layers.
# I think what's happening is that the output from the first layer is directly going into the second layer
# flat. I need to figure out how to reshape the tensors after so that they can serve as usefule input
# into sequential layers. Confirm, this is happening by simply printing the input_shape to build method.

conv_output = 32
K.set_learning_phase(1)
batch_size = 100
num_layers = 6
model = tf.keras.Sequential()
for i in range(num_layers):
    model.add(convLayer(32,adj[:100],batch_size))
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mse',
              metrics=['mae'])
model.fit(features[0:100], test[0:100],batch_size = 100)

ValueError: Tensor t0 of rank 2 does not match einsum reduction of length 3